In [0]:
from nltk.stem import SnowballStemmer
from string import punctuation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import re
import csv
import numpy as np
import pandas as pd
import operator
import sys
from textblob import TextBlob


C:\Users\HP\Anaconda4\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [0]:
redundancy_rightFormat = {
    'ckckck': 'cock',
    'fuckfuck': 'fuck',
    'fckfck':'fuck',
    'lolol': 'lol',
    'lollol': 'lol',
    'mofckas':'motherfuckers',
    'pussyfuck':'fuck',
    'gaygay': 'gay',
    'haha': 'ha',
    'sucksuck': 'suck',
    'fck':'fuck',
    'fk':'fuck',
    'fuk':'fuck',
    'fuuk':'fuck',
    'fcck':'fuck',
    'fucka':'fucker',
    'fckr':'fucker'

}
print(redundancy_rightFormat)
COMMENT_COL = 'comment_text'
ID_COL = 'id'
input_dir = './Files/'
output_dir = './Files/'

{'ckckck': 'cock', 'fuckfuck': 'fuck', 'fckfck': 'fuck', 'lolol': 'lol', 'lollol': 'lol', 'mofckas': 'motherfuckers', 'pussyfuck': 'fuck', 'gaygay': 'gay', 'haha': 'ha', 'sucksuck': 'suck', 'fck': 'fuck', 'fk': 'fuck', 'fuk': 'fuck', 'fuuk': 'fuck', 'fcck': 'fuck', 'fucka': 'fucker', 'fckr': 'fucker'}


In [0]:
redundancy = set(redundancy_rightFormat.keys())
print(redundancy)

{'mofckas', 'ckckck', 'sucksuck', 'lolol', 'gaygay', 'pussyfuck', 'haha', 'lollol', 'fckfck', 'fuckfuck'}


In [0]:
toxic_indicator_words = [
    'fuck', 'fucking', 'fucked', 'fuckin', 'fucka', 'fucker', 'fucks', 'fuckers',
    'fck', 'fcking', 'fcked', 'fckin', 'fcker', 'fcks',
    'fuk', 'fuking', 'fuked', 'fukin', 'fuker', 'fuks', 'fukers',
    'fk', 'fking', 'fked', 'fkin', 'fker', 'fks', 'ashole', 'asholes',
    'niga', 'nigga', 'nigar', 'niggar', 'niger', 'nigger',
    'monster', 'monsters',
    'loser', 'losers',
    'nazi', 'nazis',
    'cock', 'cocks', 'cocker', 'cockers','cocksucker'
    'faggot', 'faggy',
    'cum','cumming'
]
print(toxic_indicator_words)

['fuck', 'fucking', 'fucked', 'fuckin', 'fucka', 'fucker', 'fucks', 'fuckers', 'fck', 'fcking', 'fcked', 'fckin', 'fcker', 'fcks', 'fuk', 'fuking', 'fuked', 'fukin', 'fuker', 'fuks', 'fukers', 'fk', 'fking', 'fked', 'fkin', 'fker', 'fks', 'shit', 'shitty', 'shite', 'stupid', 'stupids', 'idiot', 'idiots', 'suck', 'sucker', 'sucks', 'sucka', 'sucked', 'sucking', 'ass', 'asses', 'asshole', 'assholes', 'ashole', 'asholes', 'gay', 'gays', 'niga', 'nigga', 'nigar', 'niggar', 'niger', 'nigger', 'monster', 'monsters', 'loser', 'losers', 'nazi', 'nazis', 'cock', 'cocks', 'cocker', 'cockers', 'cocksuckerfaggot', 'faggy', 'cum', 'cumming']


In [0]:
toxic_indicator_words_sets = set(toxic_indicator_words)

def _get_toxicIndicator_transformers():
    toxicIndicator_transformers = dict()
    for word in toxic_indicator_words:
        tmp_1 = []
        for c in word:
            if len(tmp_1) > 0:
                tmp_2 = []
                for pre in tmp_1:
                    tmp_2.append(pre + c)
                    tmp_2.append(pre + c + c)
                tmp_1 = tmp_2
            else:
                tmp_1.append(c)
                tmp_1.append(c + c)
        toxicIndicator_transformers[word] = tmp_1
    return toxicIndicator_transformers


toxicIndicator_transformers = _get_toxicIndicator_transformers()
print(toxicIndicator_transformers)

{'fuck': ['fuck', 'fuckk', 'fucck', 'fucckk', 'fuuck', 'fuuckk', 'fuucck', 'fuucckk', 'ffuck', 'ffuckk', 'ffucck', 'ffucckk', 'ffuuck', 'ffuuckk', 'ffuucck', 'ffuucckk'], 'fucking': ['fucking', 'fuckingg', 'fuckinng', 'fuckinngg', 'fuckiing', 'fuckiingg', 'fuckiinng', 'fuckiinngg', 'fuckking', 'fuckkingg', 'fuckkinng', 'fuckkinngg', 'fuckkiing', 'fuckkiingg', 'fuckkiinng', 'fuckkiinngg', 'fuccking', 'fucckingg', 'fucckinng', 'fucckinngg', 'fucckiing', 'fucckiingg', 'fucckiinng', 'fucckiinngg', 'fucckking', 'fucckkingg', 'fucckkinng', 'fucckkinngg', 'fucckkiing', 'fucckkiingg', 'fucckkiinng', 'fucckkiinngg', 'fuucking', 'fuuckingg', 'fuuckinng', 'fuuckinngg', 'fuuckiing', 'fuuckiingg', 'fuuckiinng', 'fuuckiinngg', 'fuuckking', 'fuuckkingg', 'fuuckkinng', 'fuuckkinngg', 'fuuckkiing', 'fuuckkiingg', 'fuuckkiinng', 'fuuckkiinngg', 'fuuccking', 'fuucckingg', 'fuucckinng', 'fuucckinngg', 'fuucckiing', 'fuucckiingg', 'fuucckiinng', 'fuucckiinngg', 'fuucckking', 'fuucckkingg', 'fuucckkinng', '

In [0]:
deny_origin = {
    "you're": ['you', 'are'],
    "i'm": ['i', 'am'],
    "he's": ['he', 'is'],
    "she's": ['she', 'is'],
    "it's": ['it', 'is'],
    "they're": ['they', 'are'],
    "can't": ['can', 'not'],
    "couldn't": ['could', 'not'],
    "don't": ['do', 'not'],
    "don;t": ['do', 'not'],
    "didn't": ['did', 'not'],
    "doesn't": ['does', 'not'],
    "isn't": ['is', 'not'],
    "wasn't": ['was', 'not'],
    "aren't": ['are', 'not'],
    "weren't": ['were', 'not'],
    "won't": ['will', 'not'],
    "wouldn't": ['would', 'not'],
    "hasn't": ['has', 'not'],
    "haven't": ['have', 'not'],
    "what's": ['what', 'is'],
    "that's": ['that', 'is'],
    }

In [0]:
print(deny_origin)

{"you're": ['you', 'are'], "i'm": ['i', 'am'], "he's": ['he', 'is'], "she's": ['she', 'is'], "it's": ['it', 'is'], "they're": ['they', 'are'], "can't": ['can', 'not'], "couldn't": ['could', 'not'], "don't": ['do', 'not'], 'don;t': ['do', 'not'], "didn't": ['did', 'not'], "doesn't": ['does', 'not'], "isn't": ['is', 'not'], "wasn't": ['was', 'not'], "aren't": ['are', 'not'], "weren't": ['were', 'not'], "won't": ['will', 'not'], "wouldn't": ['would', 'not'], "hasn't": ['has', 'not'], "haven't": ['have', 'not'], "what's": ['what', 'is'], "that's": ['that', 'is']}


In [0]:
denies = set(deny_origin.keys())

In [0]:
curse_words_variation_key={
    '@ss':'ass',
    '@$$':'ass',
    'a55':'ass',
    'dik':'dick',
    '$h1t':'shit',
    '$hit':'shit',
    '$h!t':'shit',
    'sh!t':'shit',
    'sh*t':'shit',
    'b!tch':'bitch',
    'bi+ch':'bitch',
    'c0ck':'cock',
    'c**k':'cock',
    'c*ck':'cock',
    'f*ck':'fuck',
    'fu*k':'fuck',
    'l3itch':'bitch',
    'b*tch':'bitch',
    'p*ssy':'pussy',
    'd!ck':'dick',
    'n!gga':'nigga',
    'n!gg@':'nigga'
}
cur_wrd_key=set(curse_words_variation_key.keys())

#######Curse words most commonly used//////////https://www.fastcompany.com/3026596/140-characters-of-fck-sht-and-ss-how-we-swear-on-twitter
############shit (15.04%), ass (14.48%), bitch (10.34%), nigga (9.68%), hell (4.46%), whore (1.82%), dick (1.67%), piss (1.53%), and pussy (1.16%).

In [0]:
#Using CLeanTextPreprocess we tried to provide it the filtering that wud assist our embedding to perform better 
class CleanTextPreprocess(object):
    
    #9-Drop all the wiki tokens available in the text like image , css , and templates  
    @staticmethod
    def drop_tokens(text):
        # Drop the image
        text = re.sub(r"image:[a-zA-Z0-9]*\.jpg", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.png", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.gif", " ", text)
        text = re.sub(r"image:[a-zA-Z0-9]*\.bmp", " ", text)

        # Drop css
        text = re.sub(r"#([A-Fa-f0-9]{6}|[A-Fa-f0-9]{3})", " ",text)
        text = re.sub(r"\{\|[^\}]*\|\}", " ", text)
        
        # Clean templates
        text = re.sub(r"\[?\[user:.*\]", " ", text)
        text = re.sub(r"\[?\[user:.*\|", " ", text)        
        text = re.sub(r"\[?\[wikipedia:.*\]", " ", text)
        text = re.sub(r"\[?\[wikipedia:.*\|", " ", text)
        text = re.sub(r"\[?\[special:.*\]", " ", text)
        text = re.sub(r"\[?\[special:.*\|", " ", text)
        text = re.sub(r"\[?\[category:.*\]", " ", text)
        text = re.sub(r"\[?\[category:.*\|", " ", text)
        
        return text

    
    @staticmethod
    def text_space_out_expand(t):
        t=re.sub(r"\."," . ",t)
        t=re.sub(r"\?"," ? ",t)
        t=re.sub(r"\'s", "is", t)
        t=re.sub(r"\'ve", " have ",t)
        t=re.sub(r"n't", " not ",t)
        t=re.sub(r"\'re", " are ",t)
        t=re.sub(r"\'d", " would ",t)
        t=re.sub(r"\'ll", " will ",t)
        return t
    
    #4-Correction using the textblob library for processing textual data
    @staticmethod
    def curse_typo_variant(t):
        ret=[]
        for wrd in t.split(' '):
            if wrd in cur_wrd_key:
                wrd=curse_words_variation_key[wrd]
            ret.append(wrd) 
        ret =' '.join(ret)
        return (ret)
       
    
    #3-expand the words with 's in the text
    @staticmethod
    def expand_words(t):
        ret=[]
        for word in t.split():
            if word in denies:
                ret.append(deny_origin[word][0])
                ret.append(deny_origin[word][1])
            else:
                ret.append(word)
        ret=' '.join(ret)
        ret=re.sub("'",' ',ret)
        ret=re.sub(";",' ',ret)
        return ret
    
    
    #2-chance of occurrence of word with three same consecutive alphabets in a word is rare
    @staticmethod
    def reduce_letter_more_twice(t):
        for letter in 'abcdefghijklmnopqrstuvwxyz':
            reg=letter+"{2,}"
            t=re.sub(reg,letter+letter,t)
        return t
    
    #6-replace the redundancy word 
    @staticmethod
    def replace_redundancy_words(t):
        chng=[]
        for wrd in t.split(' '):
            for red in redundancy:
                if red in wrd:
                    wrd=redundancy_rightFormat[red]
                    break
            chng.append(wrd)
        return ' '.join(chng)
    
    #7-omission of star in the word to get a version of curse words
    @staticmethod
    def star_omition(t):
        ret=[]
        for wrd in t.split(' '):
            if ('*' in wrd) and (re.sub('\*','',wrd) in toxic_indicator_words):
                wrd=re.sub('\*','',wrd)
                break
            ret.append(wrd)
        ret=" ".join(ret)
        return ret
    
#     #8-misspelled curse words correction
#     @staticmethod
#     def toxic_wrd_correction(t):
#         ret=[]
#         for wrd in t.split(' '):
#             for tox_wrd in toxic_indicator_words_sets:
#                 if (wrd in toxicIndicator_transformers[tox_wrd]):
#                     wrd=tox_wrd
#                     break;
#             ret.append(wrd)
#         return ' '.join(ret)
    
#     #10-put NA in empty comments and filter out all the special characters except A-Z ,a-z or ,?!
    @staticmethod
    def filter_out(t):
        t=re.sub(r"[^A-Za-z,!?*.,-]", " ", t)
        if t.strip() == '':
            return 'NA'
        t=TextBlob(t)
        return str(t.correct())
    
    #1-change all the character to lower case 
    @staticmethod
    def lower_case(t):
        return t.lower()
    
def exec_all_funtions(df):
    process_pipe=[
            CleanTextPreprocess.lower_case,
            CleanTextPreprocess.reduce_letter_more_twice,
            CleanTextPreprocess.expand_words,
            CleanTextPreprocess.curse_typo_variant,
            CleanTextPreprocess.text_space_out_expand,
            CleanTextPreprocess.replace_redundancy_words,
            CleanTextPreprocess.star_omition,
#             CleanTextPreprocess.toxic_wrd_correction,
            CleanTextPreprocess.drop_tokens,
            CleanTextPreprocess.filter_out
        ]
    print('\nlower_casing started')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[0])
    df.to_csv('test_lower.csv', index=False)
    print('\nlower_casing completed')
    print('\nstarted 1')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[1])
    df.to_csv('test_reduce_letter.csv', index=False)
    print("\nended 1")
    print('\nstarted 2')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[2])
    df.to_csv('test_expand.csv', index=False)
    print("\nended 2")
    print('\nstarted 3')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[3])
    df.to_csv('test_curse.csv', index=False)
    print("\nended 3")
    print('\nstarted 4')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[4])
    df.to_csv('test_space_out.csv', index=False)
    print("\nended 4")
    print('\nstarted 5')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[5])
    df.to_csv('test_red_wrd.csv', index=False)
    print("\nended 5")
    print('\nstarted 6')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[6])
    df.to_csv('test_star_omi.csv', index=False)
    print("\nended 6")
    print('\nstarted 7')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[7])
    df.to_csv('test_drop_t.csv', index=False)
    print("\nended 7")
    print('\nstarted 8')
    df[COMMENT_COL]=df[COMMENT_COL].apply(process_pipe[8])
    df.to_csv('test_filter_out.csv', index=False)
    print("\nended 8")
    print('\nFinal step done')
    return df

  
  
#################Swao among the commented and uncommented path below######################
  
# Process whole train data
# print('Comm processing whole train data')
# df_train = pd.read_csv(input_dir + 'train.csv')
# df_train = exec_all_funtions(df_train)
# df_train.to_csv('train_processed.csv', index=False)

# Process test data
print('Comm processing test data')
df_test = pd.read_csv(input_dir + 'test.csv')
df_test = exec_all_funtions(df_test)
df_test.to_csv('test_processed.csv', index=False)


In [0]:
##############Dictionary for the simileys , can be added to the projected
smiley={
  ':‑)':'happy',  
  ':-]':'happy',
  ':-3':'happy',
  ':->':'happy',
    '8-)':'happy',
    ':-}': 'happy',
    ':o)':'happy',
    ':c)':'happy',
    ':)':'happy',
    ':]': 'smiley',
    ':3':'smiley',
    ':>':'smiley',
    '8)':'smiley',
    ':^)':'smiley',
    '=]':'smiley',
    
    ':‑D':'Laughing',
    '8‑D':'Laughing',
    'x‑D':'Laughing',
    'X‑D':'Laughing',
    'B^D':'Laughing',
    '=D':'Laughing',
    '=3':'Laughing',
    '8D':'Laughing',
    'xD':'Laughing',
    ':D':'Laughing',
    ':-))':'very happy',
    
    '‑(': 'crying',
    '(': 'crying',
    
    ':‑(': 'sad',
    ':‑c':  'sad',
    ':‑<':'sad',
    ':‑[':'sad',
    ':-||':'sad',
    '>:[':'sad',
    ':{':'sad',
    ':@':'sad',
    '>:(':'sad',
    
  'D:<': 'disgust',
    'D:':'disgust',
    'D8':'disgust',
    'D=':'disgust',
    'DX':'disgust',
    
    ':‑O':'surprise',
    ':O':'surprise',
    ':o':'surprise',
    '8‑0':'surprise',
    '>:O':'surprise',
    
    ':$':'blushing',
    ':$':'Embarrassed',
    
    '%‑)':'drunk',
    ':‑###..':'Being sick',
    '<:‑|':'dumb',
    '>_>':'guilty',
    
    '|‑O':'bored',
    'v.v':'horror',
    
    
    
    '(=^·^=)':'cat',
    '(._.)':'looking down',
    '(-.-)y-°°°':'smoking',
    '(-_-)zzz':'sleeping',
    '(+o+)':'confused',
    '(;_:)':'sad',
    '(ー_ー)!!': 'shame',
    '(一一)':'shame',
    '(-.-)':'shame',
    '(=_=)':'tired',
    '>^_^<':'Normal laugh',
    '(*^.^*)':'Normal laugh',
    '(^.^)':'Normal laugh',
    '(*_*)':'Amazed',
    '(⌒▽⌒）':'Laughing',
    '(-"-)':'worried',
    '(-_-メ)':'worried',
    '(°◇°)':'shocked',
    '(´；ω；`)':'terribly sad',
    'Σ(゜д゜;)':'shocked',
    '(´ー`)y-~~':'smoking',
    '（ つ Д ｀）':'sad',
    
    '👎🏾': ''
    '😡':'pouting face',
    '🤬':'swearing',
    '😠':'angry face',
    '💩':'smiling poo',
    '👋🏽':'hello ',
    '🖕':'insulting',
    ''
    
    
    
    
}